In [1]:
import os

import numpy as np
import pandas as pd
from pathlib import Path

from flip import fisher, utils, data_vector, __flip_dir_path__
from flip.covariance import covariance


flip_base = Path(__flip_dir_path__)
data_path = flip_base / 'data'

[ 000000.00 ]: 06-09 10:34  root            INFO     No pypower module detected, gridding with this method is unavailable
[ 000000.07 ]: 06-09 10:34  jax._src.xla_bridge INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[ 000000.07 ]: 06-09 10:34  jax._src.xla_bridge INFO     Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [2]:
sn_data = pd.read_parquet(data_path / "velocity_data.parquet")

In [3]:
sn_data = sn_data[np.array(sn_data["status"]) != False]
sn_data = sn_data[np.array(sn_data["status"]) != None]

In [4]:
sn_data["dmu_error"] = np.random.normal(0, 0.1, len(sn_data))

In [5]:
DataFisher = data_vector.FisherVelFromHDres(sn_data.to_dict(orient='list'))
print("Data free parameters: ", DataFisher.free_par)


Data free parameters:  ['sigma_M']


In [6]:
ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")
sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}

### Compute covariance
size_batch = 10_000
number_worker = 16

covariance_fit = DataFisher.compute_covariance(
    "rcrk24",
    power_spectrum_dict,
    size_batch=size_batch,
    number_worker=number_worker,
    variant = 'growth_index'
)
print("Covariance free parameters: ", covariance_fit.free_par)


/hpc/group/cosmology/blc56/miniconda3/envs/flip/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000000.72 ]: 06-09 10:34  root            INFO     Covariance matrix generated from flip with rcrk24 model in 1.78e+00 seconds
[ 000000.74 ]: 06-09 10:34  root            INFO     Use self.init_covariance_sum before using self.compute_covariance_sum.


Covariance free parameters:  ['Om0', 'gamma', 'sigv']


In [7]:
fisher_properties = {
    "inversion_method": "inverse",
}


In [8]:
parameter_dict = {
    "gamma": 0.55,
    "Om0": 0.3,
    "sigv": 200,
    "sigma_M": 0.12
}

Fisher = fisher.FisherMatrix.init_from_covariance(
    covariance_fit,
    DataFisher,
    parameter_dict,
    fisher_properties=fisher_properties,
)

parameter_name_list, fisher_matrix = Fisher.compute_fisher_matrix()

ERR = np.sqrt(np.diag(np.linalg.pinv(fisher_matrix)))

In [9]:
for p, e in zip(parameter_name_list, ERR):
    print(f"Fisher error on {p} = {e:.4f}")

Fisher error on Omegam = 5.6934
Fisher error on gamma = 9.6916
